# Schedule an endpoint

In [5]:
# Get the endpoint name from the AzureML studio in the Pipelines -> Pipeline endpoints
pipeline_endpoint_name = "forecast-with-r"

In [3]:
import azureml

from azureml.core import Workspace

print("Azure ML SDK Version: ", azureml.core.VERSION)
# Connect to Workspace
ws = Workspace.from_config()

Azure ML SDK Version:  1.35.0


In [6]:
from azureml.pipeline.core import PipelineEndpoint, Schedule 

pipeline_endpoint = PipelineEndpoint.get(workspace=ws, name=pipeline_endpoint_name)


In [7]:
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule, TimeZone
from datetime import datetime

recurrence = ScheduleRecurrence(frequency="Day", 
                                interval=1, 
                                hours=[22], 
                                minutes=[30],
                                time_zone=TimeZone.EEuropeStandardTime,
                                start_time = datetime.now()  # This will prevent starting the pipeline immediately after scheduling it
                                ) # Runs every other day at 10:30pm Eastern European Standard Time (aka Greece)

schedule = Schedule.create_for_pipeline_endpoint(workspace=ws, name=f"{pipeline_endpoint_name}-schedule",
                           pipeline_endpoint_id=pipeline_endpoint.id, 
                           pipeline_parameters = {  # These are the parameters that will be passed to the pipeline
                               "start_date": "2020-01-01",
                               "key_vault": "my-key-vault",
                           },
                           experiment_name=f"{pipeline_endpoint_name}-schedule-run",
                           recurrence=recurrence,
                           wait_for_provisioning=True,
                           description="Scheduled pipeline run")

# You may want to make sure that the schedule is provisioned properly
# before making any further changes to the schedule

print("Created schedule with id: {}".format(schedule.id))

Provisioning status: Completed
Created schedule with id: d3fa9aa6-cf31-4d80-9152-becf4360ff20


In [13]:
from azureml.pipeline.core.schedule import Schedule
# Use active_only=False to get all schedules including disabled schedules
schedules = Schedule.list(ws, active_only=True) 
print("Your workspace has the following schedules set up:")
for schedule in schedules:
    print("Schedule Id: {} (Published pipeline: {}, Published endpoint: {})".format(schedule.id, schedule.pipeline_id, schedule.pipeline_endpoint_id))
    if schedule.id == "a specific schedule id you want to disable":
      schedule.disable(wait_for_provisioning=True)

Your workspace has the following schedules set up:
Schedule Id: 79e8a585-4e9d-4060-849e-342553f108fd (Published pipeline: 3855df31-cff6-4341-9707-6841abc862fb, Published endpoint: None)
